# 🚚 Sistema de Simulación Logística - Andes Logistics S.A.
## Ferreyros - Simulación de Desempeño Logístico

**Objetivo:** Simular, consolidar y analizar el desempeño logístico en operaciones de distribución de repuestos Caterpillar.

**Versión:** 1.0  
**Fecha:** Noviembre 2025  
**Ciclo:** SEPTIMO - Sistemas Integrados Empresariales

## 1️⃣ Importar Librerías y Configuración Inicial

In [ ]:
import sys
import os
import json
import random
from datetime import datetime, timedelta
import pandas as pd

# Agregar el directorio del sistema al path
sys.path.insert(0, os.path.join(os.getcwd(), '..'))

# Importar módulos del sistema
from sistema.catalogos import dic_sku, dic_clientes, dic_vehiculos, distancias_km
from sistema.demanda import simular_demanda, contar_unidades_pedidos, obtener_sku_mas_solicitado
from sistema.inventario import inicializar_stock, reservar_y_actualizar, reponer_simple, obtener_estado_stock
from sistema.picking import asignar_picking, calcular_productividad_picking
from sistema.transporte import planificar_rutas
from sistema.indicadores import calcular_indicadores, consolidar_indicadores_multiples_dias
from sistema.alertas import generar_alertas, generar_recomendaciones
from sistema.reporte import reporte_logistica, formatear_reporte_texto, exportar_reporte_csv

# Configuración global del sistema
SEED_SIMULACION = 42  # Para reproducibilidad
CAPACIDAD_PICKING_DIARIA = 1500  # unidades
HORAS_JORNADA = 8
PUNTO_REORDEN = 50
LOTE_REPOSICION = 100
STOCK_INICIAL = 200

# Establecer seed
random.seed(SEED_SIMULACION)

print("✓ Librerías importadas correctamente")
print(f"✓ Seed configurado: {SEED_SIMULACION}")
print(f"✓ Capacidad de picking: {CAPACIDAD_PICKING_DIARIA} unidades/día")

## 2️⃣ Definir Catálogos Base (SKUs, Clientes, Vehículos)

In [ ]:
# Mostrar catálogos del sistema
print("=" * 70)
print("CATÁLOGO DE SKUs (Repuestos Caterpillar)")
print("=" * 70)
df_skus = pd.DataFrame([
    {"Código": sku, "Descripción": desc}
    for sku, desc in dic_sku.items()
])
print(df_skus.to_string(index=False))

print("\n" + "=" * 70)
print("CATÁLOGO DE CLIENTES")
print("=" * 70)
df_clientes = pd.DataFrame([
    {"Código": cid, "Nombre": nombre}
    for cid, nombre in dic_clientes.items()
])
print(df_clientes.to_string(index=False))

print("\n" + "=" * 70)
print("CATÁLOGO DE VEHÍCULOS")
print("=" * 70)
df_vehiculos = pd.DataFrame([
    {
        "Código": vid,
        "Tipo": info["tipo"],
        "Capacidad (unid)": info["capacidad"],
        "Costo/km (S/)": f"{info['costo_km']:.2f}"
    }
    for vid, info in dic_vehiculos.items()
])
print(df_vehiculos.to_string(index=False))

print("\n✓ Catálogos cargados correctamente")

## 3️⃣ Simular Demanda de Pedidos

In [ ]:
# Simular demanda para 3 días
N_DIAS = 3
print(f"\n{'=' * 70}")
print(f"SIMULACIÓN DE DEMANDA - {N_DIAS} DÍAS")
print(f"{'=' * 70}\n")

pedidos_simulados = simular_demanda(N_DIAS, dic_clientes, dic_sku, seed=SEED_SIMULACION)

# Resumen diario
resumen_demanda = []
for dia in range(1, N_DIAS + 1):
    pedidos_dia = pedidos_simulados[dia]
    num_pedidos = len(pedidos_dia)
    unidades_totales = contar_unidades_pedidos(pedidos_dia)
    sku_popular, cantidad_sku = obtener_sku_mas_solicitado(pedidos_dia)
    
    resumen_demanda.append({
        "Día": dia,
        "Pedidos": num_pedidos,
        "Unidades Totales": unidades_totales,
        "SKU Popular": sku_popular,
        "Cantidad (unid)": cantidad_sku
    })
    
    print(f"📅 DÍA {dia}:")
    print(f"   - Pedidos recibidos: {num_pedidos}")
    print(f"   - Unidades solicitadas: {unidades_totales}")
    print(f"   - SKU más solicitado: {sku_popular} ({cantidad_sku} unidades)")
    print()

# Tabla consolidada
df_demanda = pd.DataFrame(resumen_demanda)
print("RESUMEN CONSOLIDADO DE DEMANDA:")
print(df_demanda.to_string(index=False))

# Totales
total_pedidos = df_demanda["Pedidos"].sum()
total_unidades = df_demanda["Unidades Totales"].sum()
promedio_diario = total_unidades / N_DIAS

print(f"\n{'─' * 70}")
print(f"Total período: {total_pedidos} pedidos | {total_unidades} unidades")
print(f"Promedio diario: {promedio_diario:.0f} unidades")
print(f"{'─' * 70}")

## 4️⃣ Gestionar Inventario y Reposición

In [ ]:
# Inicializar stock
stock = inicializar_stock(dic_sku, STOCK_INICIAL)

print(f"\n{'=' * 70}")
print("GESTIÓN DE INVENTARIO")
print(f"{'=' * 70}\n")

print(f"Stock inicial (200 unidades por SKU):")
estado_inicial = obtener_estado_stock(stock, dic_sku)
for sku, info in list(estado_inicial.items())[:3]:
    print(f"  {sku}: {info['cantidad']} unidades")
print("  ...")

# Procesar pedidos por día y gestionar inventario
historial_inventario = []
total_unidades_entregadas = 0
total_unidades_no_entregadas = 0

for dia in range(1, N_DIAS + 1):
    print(f"\n{'─' * 70}")
    print(f"📅 PROCESAMIENTO DÍA {dia}")
    print(f"{'─' * 70}")
    
    # Reservar y actualizar
    stock, entregadas, no_entregadas, log = reservar_y_actualizar(
        stock, pedidos_simulados[dia], dic_clientes
    )
    
    total_unidades_entregadas += entregadas
    total_unidades_no_entregadas += no_entregadas
    
    print(f"Unidades entregadas: {entregadas}")
    print(f"Unidades no entregadas: {no_entregadas}")
    
    # Reposición automática
    stock, log_reposicion = reponer_simple(stock, dic_sku, PUNTO_REORDEN, LOTE_REPOSICION)
    
    if log_reposicion:
        print(f"Reposiciones realizadas: {len(log_reposicion)}")
        for rep in log_reposicion[:2]:
            print(f"  - {rep['sku']}: {rep['cantidad_añadida']} unidades añadidas")
    else:
        print("Sin reposiciones necesarias")
    
    # Guardar estado
    historial_inventario.append({
        "Día": dia,
        "Entregadas": entregadas,
        "No Entregadas": no_entregadas,
        "Reposiciones": len(log_reposicion)
    })

# Mostrar stock final
print(f"\n{'─' * 70}")
print("Stock final después de 3 días:")
estado_final = obtener_estado_stock(stock, dic_sku)
for sku, info in list(estado_final.items())[:3]:
    print(f"  {sku}: {info['cantidad']} unidades")
print("  ...")

print(f"\nTotal entregado: {total_unidades_entregadas} unidades")
print(f"Total no entregado: {total_unidades_no_entregadas} unidades")

## 5️⃣ Ejecutar Operaciones de Picking

In [ ]:
# Ejecutar operaciones de picking por día
print(f"\n{'=' * 70}")
print("OPERACIONES DE PICKING")
print(f"{'=' * 70}\n")

resultados_picking = []
for dia in range(1, N_DIAS + 1):
    picking = asignar_picking(dia, pedidos_simulados[dia], CAPACIDAD_PICKING_DIARIA)
    
    print(f"{'─' * 70}")
    print(f"📦 DÍA {dia} - ASIGNACIÓN DE PICKING")
    print(f"{'─' * 70}")
    print(f"Capacidad disponible: {picking['capacidad_disponible']:,} unidades")
    print(f"Capacidad utilizada: {picking['capacidad_usada']:,} unidades ({picking['capacidad_usada']/picking['capacidad_disponible']*100:.1f}%)")
    print(f"Pedidos preparados: {picking['num_pedidos_preparados']}")
    print(f"Pedidos pendientes: {picking['num_pedidos_pendientes']} (Backlog: {picking['unidades_pendientes']:,} unidades)")
    
    # Calcular productividad
    productividad = calcular_productividad_picking(picking['unidades_preparadas'], HORAS_JORNADA)
    print(f"Productividad: {productividad:.2f} unidades/hora\n")
    
    resultados_picking.append({
        "Día": dia,
        "Preparados": picking['num_pedidos_preparados'],
        "Pendientes": picking['num_pedidos_pendientes'],
        "Unid. Preparadas": picking['unidades_preparadas'],
        "Unid. Pendientes": picking['unidades_pendientes'],
        "Prod. (unid/h)": productividad
    })

# Tabla de resultados
df_picking = pd.DataFrame(resultados_picking)
print("RESUMEN DE PICKING:")
print(df_picking.to_string(index=False))

# Guardar resultados para paso siguiente
picking_3dias = []
for dia in range(1, N_DIAS + 1):
    picking_3dias.append(asignar_picking(dia, pedidos_simulados[dia], CAPACIDAD_PICKING_DIARIA))

## 6️⃣ Planificar Rutas de Transporte

In [ ]:
# Planificar rutas de transporte
print(f"\n{'=' * 70}")
print("PLANIFICACIÓN DE RUTAS DE TRANSPORTE")
print(f"{'=' * 70}\n")

resultados_transporte = []
for dia in range(1, N_DIAS + 1):
    # Usar pedidos preparados del picking
    pedidos_preparados = picking_3dias[dia-1]['preparados']
    
    rutas = planificar_rutas(dia, pedidos_preparados, dic_vehiculos, distancias_km, dic_clientes)
    
    print(f"{'─' * 70}")
    print(f"🚚 DÍA {dia} - PLANIFICACIÓN DE RUTAS")
    print(f"{'─' * 70}")
    print(f"Rutas planificadas: {rutas['num_rutas']}")
    print(f"Unidades transportadas: {rutas['unidades_transportadas']:,}")
    print(f"Utilización promedio de flota: {rutas['utilizacion_promedio']:.1f}%")
    print(f"Costo total estimado: S/. {rutas['costo_total']:.2f}\n")
    
    # Detalles de rutas
    if rutas['rutas']:
        print("Detalle de rutas:")
        for i, ruta in enumerate(rutas['rutas'][:3], 1):
            print(f"  {i}. Vehículo {ruta['vehiculo']} → {ruta['cliente']}: {ruta['unidades']} unid ({ruta['utilizacion']:.1f}%)")
        if len(rutas['rutas']) > 3:
            print(f"  ... + {len(rutas['rutas']) - 3} rutas más")
    
    resultados_transporte.append({
        "Día": dia,
        "Rutas": rutas['num_rutas'],
        "Unid. Transportadas": rutas['unidades_transportadas'],
        "Util. Flota (%)": rutas['utilizacion_promedio'],
        "Costo Total (S/)": rutas['costo_total']
    })

# Tabla de resultados
df_transporte = pd.DataFrame(resultados_transporte)
print("\nRESUMEN DE TRANSPORTE:")
print(df_transporte.to_string(index=False))

## 7️⃣ Calcular Indicadores Logísticos

In [ ]:
# Calcular indicadores por día
print(f"\n{'=' * 70}")
print("INDICADORES LOGÍSTICOS (KPIs)")
print(f"{'=' * 70}\n")

lista_indicadores_diarios = []
for dia in range(1, N_DIAS + 1):
    picking = picking_3dias[dia-1]
    pedidos_dia = pedidos_simulados[dia]
    
    # Totales del día
    unidades_solicitadas = contar_unidades_pedidos(pedidos_dia)
    unidades_preparadas = picking['unidades_preparadas']
    unidades_pendientes_picking = picking['unidades_pendientes']
    
    # Obtener transporte
    pedidos_preparados = picking['preparados']
    rutas = planificar_rutas(dia, pedidos_preparados, dic_vehiculos, distancias_km, dic_clientes)
    unidades_transportadas = rutas['unidades_transportadas']
    utilizacion_flota = rutas['utilizacion_promedio']
    
    # Calcular indicadores
    indicadores = calcular_indicadores(
        len(pedidos_dia),
        unidades_transportadas,
        unidades_solicitadas,
        unidades_preparadas,
        unidades_transportadas,
        unidades_pendientes_picking,
        utilizacion_flota,
        HORAS_JORNADA
    )
    
    lista_indicadores_diarios.append(indicadores)
    
    print(f"{'─' * 70}")
    print(f"📊 INDICADORES DÍA {dia}")
    print(f"{'─' * 70}")
    print(f"OTIF: {indicadores['otif']:.2f}%")
    print(f"Fill Rate: {indicadores['fill_rate']:.2f}%")
    print(f"Backlog Rate: {indicadores['backlog_rate']:.2f}%")
    print(f"Productividad Picking: {indicadores['productividad_picking']:.2f} unid/h")
    print(f"Utilización de Flota: {indicadores['utilizacion_flota']:.2f}%")
    print(f"Índice de Transporte: {indicadores['indice_transporte']:.2f}%")
    print()

# Consolidar indicadores
indicadores_consolidados = consolidar_indicadores_multiples_dias(lista_indicadores_diarios)

print(f"\n{'=' * 70}")
print("INDICADORES CONSOLIDADOS (3 DÍAS)")
print(f"{'=' * 70}")
print(f"OTIF promedio: {indicadores_consolidados['otif_promedio']:.2f}%")
print(f"Fill Rate promedio: {indicadores_consolidados['fill_rate_promedio']:.2f}%")
print(f"Backlog Rate promedio: {indicadores_consolidados['backlog_rate_promedio']:.2f}%")
print(f"Productividad promedio: {indicadores_consolidados['productividad_picking_promedio']:.2f} unid/h")
print(f"Utilización de flota promedio: {indicadores_consolidados['utilizacion_flota_promedio']:.2f}%")
print(f"Total entregado: {indicadores_consolidados['unidades_entregadas_total']:,} unidades")
print(f"Total no entregado: {indicadores_consolidados['unidades_no_entregadas_total']:,} unidades")

## 8️⃣ Generar Alertas Automáticas

In [ ]:
# Generar alertas basadas en indicadores consolidados
print(f"\n{'=' * 70}")
print("ALERTAS AUTOMÁTICAS DEL SISTEMA")
print(f"{'=' * 70}\n")

# Usar los indicadores consolidados para generar alertas
alertas = generar_alertas(indicadores_consolidados)

if alertas:
    for i, alerta in enumerate(alertas, 1):
        severidad_emoji = {"ALTO": "🔴", "MEDIO": "🟡", "BAJO": "🟢"}
        emoji = severidad_emoji.get(alerta['severidad'], "⚪")
        print(f"{emoji} [{alerta['severidad']}] {alerta['tipo']}")
        print(f"   Mensaje: {alerta['mensaje']}")
        print(f"   Recomendación: {alerta['recomendacion']}")
        print()
else:
    print("✅ No hay alertas activas. Sistema operando dentro de parámetros normales.\n")

# Generar recomendaciones
recomendaciones = generar_recomendaciones(alertas, indicadores_consolidados)

print(f"{'=' * 70}")
print("RECOMENDACIONES AUTOMÁTICAS")
print(f"{'=' * 70}\n")

if recomendaciones:
    for i, rec in enumerate(recomendaciones, 1):
        print(f"{i}. {rec}")
else:
    print("No hay recomendaciones adicionales en este momento.")

## 9️⃣ Producir Reporte Final Consolidado

In [ ]:
# Generar reporte final consolidado
print("\n\n")
reporte_final = reporte_logistica(
    df_demanda["Pedidos"].sum(),
    df_demanda["Unidades Totales"].sum(),
    indicadores_consolidados['unidades_entregadas_total'],
    indicadores_consolidados,
    alertas,
    recomendaciones
)

# Mostrar reporte formateado
texto_reporte = formatear_reporte_texto(reporte_final)
print(texto_reporte)

# Guardar reporte en archivo
with open('../data/reporte_final.txt', 'w', encoding='utf-8') as f:
    f.write(texto_reporte)

print("\n✓ Reporte guardado en: data/reporte_final.txt")

# Exportar a CSV
csv_reporte = exportar_reporte_csv(reporte_final)
with open('../data/reporte_final.csv', 'w', encoding='utf-8') as f:
    f.write(csv_reporte)

print("✓ Reporte CSV guardado en: data/reporte_final.csv")

# Guardar datos detallados en JSON
datos_json = {
    "fecha_simulacion": datetime.now().isoformat(),
    "periodo_dias": N_DIAS,
    "resumen": reporte_final["resumen_operaciones"],
    "indicadores": reporte_final["indicadores_globales"],
    "alertas": [{"tipo": a["tipo"], "severidad": a["severidad"]} for a in reporte_final["alertas"]],
    "recomendaciones": reporte_final["recomendaciones"]
}

with open('../data/simulacion.json', 'w', encoding='utf-8') as f:
    json.dump(datos_json, f, indent=4, ensure_ascii=False)

print("✓ Datos de simulación guardados en: data/simulacion.json")

## 🔟 Visualizar Resultados y Métricas

In [ ]:
# Crear visualizaciones de resultados
print("\n" + "=" * 70)
print("VISUALIZACIÓN DE MÉTRICAS CONSOLIDADAS")
print("=" * 70 + "\n")

# Tabla consolidada de demanda
print("📊 EVOLUCIÓN DE DEMANDA:")
print(df_demanda.to_string(index=False))
print()

# Tabla consolidada de picking
print("📦 EVOLUCIÓN DE PICKING:")
print(df_picking.to_string(index=False))
print()

# Tabla consolidada de transporte
print("🚚 EVOLUCIÓN DE TRANSPORTE:")
print(df_transporte.to_string(index=False))
print()

# DataFrame de indicadores diarios
df_indicadores = pd.DataFrame([
    {
        "Día": i+1,
        "OTIF (%)": ind["otif"],
        "Fill Rate (%)": ind["fill_rate"],
        "Backlog (%)": ind["backlog_rate"],
        "Prod. (unid/h)": ind["productividad_picking"],
        "Flota (%)": ind["utilizacion_flota"]
    }
    for i, ind in enumerate(lista_indicadores_diarios)
])

print("📈 INDICADORES DIARIOS:")
print(df_indicadores.to_string(index=False))
print()

# Resumen ejecutivo
print("=" * 70)
print("RESUMEN EJECUTIVO")
print("=" * 70)
print(f"\n✓ Simulación completada exitosamente")
print(f"  Período: {N_DIAS} días")
print(f"  Total pedidos: {reporte_final['resumen_operaciones']['pedidos_recibidos']}")
print(f"  Total solicitado: {reporte_final['resumen_operaciones']['unidades_solicitadas']:,} unidades")
print(f"  Total entregado: {reporte_final['resumen_operaciones']['unidades_entregadas']:,} unidades ({reporte_final['indicadores_globales']['fill_rate_promedio']:.1f}%)")
print(f"  OTIF promedio: {reporte_final['indicadores_globales']['otif_promedio']:.1f}%")
print(f"  Alertas activas: {len(alertas)}")
print(f"\n✓ Archivos generados:")
print(f"  - data/reporte_final.txt")
print(f"  - data/reporte_final.csv")
print(f"  - data/simulacion.json")